In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 137kB/s 
     |████████████████████████████████| 983kB 47.1MB/s 
     |████████████████████████████████| 245kB 49.8MB/s 
     |████████████████████████████████| 1.3MB 45.7MB/s 
     |████████████████████████████████| 471kB 44.6MB/s 
     |████████████████████████████████| 2.9MB 50.7MB/s 
     |████████████████████████████████| 1.1MB 44.4MB/s 
     |████████████████████████████████| 890kB 50.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.23.1-cp36-none-any.whl size=25271896 sha256=054a324d5e3579b5452ebfd6fd0df784f252dd0949a9cf678881d3f8a536c055
  Stored in directory: /root/.cache/pip/wheels/48/13/4f/fa051ad74b07760a6f4807bac542e18081676937da6f885a0d
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=23389c70daf01eb052d16ce8c860e4fc21cd772a2cb99eeec273a81e6fe23d24
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [2]:
import matplotlib.pyplot as plt
import tensorflow as tf
from ktrain import text
import pandas as pd
import numpy as np
import ktrain
import nltk
import re

In [3]:
def get_data(url):
    f = open(url, 'r', encoding = "UTF-8")
    text = f.read()
    text_array = text.split('\n')
    text_dict = {'text':[], 'emotion':[]}
    emotion_dict = {"anger": 1, "joy":2, "fear":3, "sadness":4, "love":5, "surprise":6}
    for sentence in text_array:
        if ';' in sentence:
            a = sentence.split(';')
            text_dict['text'].append(a[0])
            text_dict['emotion'].append(a[1])
    df = pd.DataFrame(text_dict)
    return(df)

In [4]:
train = get_data('train.txt')

In [5]:
print("Shape of train is", train.shape)
train.head()

Shape of train is (16000, 2)


,text,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [6]:
(X_train, y_train), (X_test, y_test), preprocess = text.texts_from_df(train_df= train, 
                                                                      text_column = 'text',
                                                                      label_columns = 'emotion',
                                                                      maxlen = 50,
                                                                      preprocess_mode = 'bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [7]:
X_train[0].shape

(14400, 50)

In [8]:
model = text.text_classifier(name= 'bert',
                             train_data= (X_train, y_train),
                             preproc = preprocess)

Is Multi-Label? False
maxlen is 50
done.


In [9]:
# Get Learning Rate
learner = ktrain.get_learner(model = model,
                             train_data = (X_train, y_train),
                             val_data = (X_test, y_test),
                             batch_size = 32)

In [10]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)



begin training using onecycle policy with max lr of 2e-05...
450/450 [==============================] - 187s 416ms/step - loss: 0.8416 - accuracy: 0.6876 - val_loss: 0.2478 - val_accuracy: 0.9075


In [11]:
predictor = ktrain.get_predictor(learner.model, preproc= preprocess)

In [12]:
data = ["I am pretty sad and I don't have anyone to talk to.", 
        "I am great actually. Very happy with my life for now",
        "I am really scared of what is going to happen to me now"]

In [13]:
predictor.predict(data)

['sadness', 'joy', 'fear']

In [14]:
data = ["Not great. I feel suicidal and I have no one to talk to.",
        "Happy, optimistic, inspired, enthusiastic, upbeat, silly, joyful!!",
        "I feel tired, sore, and lonely. I just wan't somebody to hold.",
        "I got in trouble at work and I feel this way. like i'm going to lose my job tomorrow"]

In [15]:
predictor.predict(data)

['sadness', 'joy', 'sadness', 'sadness']